In [1]:
import urllib.request
import pickle
from os import path
import numpy as np

In [2]:
PK_PATH = '/run/media/backman/yay/imdb/imdb_full.pkl'

In [3]:
if not path.isfile(PK_PATH):
    urllib.request.urlretrieve('https://s3.amazonaws.com/text-datasets/imdb_full.pkl', PK_PATH)

In [4]:
f = open(PK_PATH, 'rb')
(x_train, labels_train), (x_test, labels_test) = pickle.load(f)

In [5]:
len(x_train)

25000

In [6]:
import tensorflow as tf

In [7]:
idx2word = tf.keras.datasets.imdb.get_word_index()

In [8]:
idx2word = {v: k for k, v in idx2word.items()}

In [9]:
idx2word[23022]

'bromwell'

In [14]:
labels_train

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


In [10]:
' '.join([idx2word[o] for o in x_train[0]])

"bromwell high is a cartoon comedy it ran at the same time as some other programs about school life such as teachers my 35 years in the teaching profession lead me to believe that bromwell high's satire is much closer to reality than is teachers the scramble to survive financially the insightful students who can see right through their pathetic teachers' pomp the pettiness of the whole situation all remind me of the schools i knew and their students when i saw the episode in which a student repeatedly tried to burn down the school i immediately recalled at high a classic line inspector i'm here to sack one of your teachers student welcome to bromwell high i expect that many adults of my age think that bromwell high is far fetched what a pity that it isn't"

In [11]:
vocab_size = 5000

trn = [np.array([i if i<vocab_size-1 else vocab_size-1 for i in s]) for s in x_train]
test = [np.array([i if i<vocab_size-1 else vocab_size-1 for i in s]) for s in x_test]

In [27]:
tf.reset_default_graph()
x_inputs = tf.placeholder(tf.int32, shape=[None])
y_labels = tf.placeholder(tf.float32, shape=[None,1])
embeddings = tf.Variable(
    tf.random_uniform([500, 32], -1.0, 1.0))
print(embeddings)
embeddings = tf.nn.embedding_lookup(embeddings, x_inputs)
print(embeddings)
embeddings = tf.reshape(embeddings, [-1,16000])
print(embeddings)
embeddings = tf.layers.dense(
                    embeddings,
                    100,
                    activation=tf.nn.relu,
                    name='dense1',
                )
embeddings = tf.layers.dropout(embeddings, 0.7, name="dense_dropout")
print(embeddings)
embeddings = tf.layers.dense(
                    embeddings,
                    1,
                    activation=tf.nn.sigmoid,
                    name='dense2',
                )
print(embeddings)


<tf.Variable 'Variable:0' shape=(500, 32) dtype=float32_ref>
Tensor("embedding_lookup:0", shape=(?, 32), dtype=float32)
Tensor("Reshape:0", shape=(?, 16000), dtype=float32)
Tensor("dense_dropout/Identity:0", shape=(?, 100), dtype=float32)
Tensor("dense2/Sigmoid:0", shape=(?, 1), dtype=float32)


In [29]:
 loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=embeddings, labels=y_labels))

In [32]:
optimizer = tf.train.AdamOptimizer(0.01).minimize(loss)

In [33]:
sess = tf.Session()

In [35]:
sess.run(tf.global_variables_initializer())


In [54]:
print(trn[0].shape)
labels_train = np.reshape(labels_train, [-1, 1])
feed_dict = {x_inputs: trn, y_labels: labels_train}
res = sess.run([optimizer, loss], feed_dict=feed_dict)

(138,)


ValueError: setting an array element with a sequence.